# Recurrent Neural Networks (RNN)

In [284]:
# data

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import models 
from keras import layers
import pandas as pd

zip_path = keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

temp=df['T (degC)']

split_fraction = 0.8
train_split = int(split_fraction * int(df.shape[0]))

step = 6 # 1시간 마다
past = 240 # 10 일
tomo = 24 # 24 시간

temp=np.array(df['T (degC)'])
temp = temp[::step]
train_split=(int)(0.7*len(temp))

data = (temp[:train_split]-np.mean(temp[:train_split]))/np.std(temp[:train_split])
test = (temp[train_split:]-np.mean(temp[train_split:]))/np.std(temp[train_split:])

def gent(ts, lookback):
    X, Y = [], []
    for i in range(len(ts) // lookback):
        X.append(ts[i*lookback:(i+1)*lookback],)
        Y.append(ts[(i+1)*lookback])
    return np.array(X), np.array(Y)

gen=gent(data,past)

In [285]:
X_train = np.reshape(gen[0], (len(gen[0]), gen[0].shape[1], 1))
y=gen[1]

gen=gent(test,past)
X_test = np.reshape(gen[0], (len(gen[0]), gen[0].shape[1], 1))
y_test=gen[1]

In [286]:
X_train.shape, y.shape

((204, 240, 1), (204,))

In [287]:
rnn = models.Sequential()
rnn.add(layers.SimpleRNN(32, activation='relu', 
                         input_shape = (240,1)))
rnn.add(layers.Dense(1))
rnn.compile(optimizer='adam',loss='mse',metrics=['acc'])

In [288]:
rnn.summary()

Model: "sequential_62"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_32 (SimpleRNN)    (None, 32)                1088      
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 33        
Total params: 1,121
Trainable params: 1,121
Non-trainable params: 0
_________________________________________________________________


In [291]:
rnn.fit(X_train,y,epochs=30,verbose=0)

In [292]:
y1=rnn.predict(X_test).flatten()
y2=y_test

np.mean(np.abs(y1-y2)), np.mean(np.abs(y1-y2))*np.std(temp)

(0.07900805795410441, 0.6655482180077538)

# Long Short-Term Memory (LSTM) 

In [322]:
lstm = models.Sequential()
lstm.add(layers.LSTM(32, activation='tanh', input_shape = (240,1)))
lstm.add(layers.Dense(1))
lstm.compile(optimizer='nadam',loss='mse',metrics=['acc'])

In [323]:
from sklearn.preprocessing import MinMaxScaler

gen=gent(data,past)

x=gen[0]
scaler=MinMaxScaler(feature_range=(0,1))
scaler.fit(x)
x=scaler.transform(x)

X_train = np.reshape(x, (len(x), x.shape[1], 1))
y=gen[1]

gen=gent(test,past)
x=gen[0]
scaler=MinMaxScaler(feature_range=(0,1))
scaler.fit(x)
x=scaler.transform(x)

X_test = np.reshape(x, (len(x), x.shape[1], 1))
y_test=gen[1]

In [324]:
lstm.fit(X_train,y,epochs=30)

Epoch 1/30
7/7 [==============================] - 0s 34ms/step - loss: 0.7964 - acc: 0.0000e+00
Epoch 2/30
7/7 [==============================] - 0s 32ms/step - loss: 0.7440 - acc: 0.0000e+00
Epoch 3/30
7/7 [==============================] - 0s 32ms/step - loss: 0.6635 - acc: 0.0000e+00
Epoch 4/30
7/7 [==============================] - 0s 32ms/step - loss: 0.3875 - acc: 0.0000e+00
Epoch 5/30
7/7 [==============================] - 0s 32ms/step - loss: 0.2302 - acc: 0.0000e+00
Epoch 6/30
7/7 [==============================] - 0s 32ms/step - loss: 0.1864 - acc: 0.0000e+00
Epoch 7/30
7/7 [==============================] - 0s 32ms/step - loss: 0.2007 - acc: 0.0000e+00
Epoch 8/30
7/7 [==============================] - 0s 32ms/step - loss: 0.1557 - acc: 0.0000e+00
Epoch 9/30
7/7 [==============================] - 0s 32ms/step - loss: 0.1427 - acc: 0.0000e+00
Epoch 10/30
7/7 [==============================] - 0s 32ms/step - loss: 0.1562 - acc: 0.0000e+00
Epoch 11/30
7/7 [======================

In [325]:
y1=lstm.predict(X_test).flatten()
y2=y_test

np.mean(np.abs(y1-y2)), np.mean(np.abs(y1-y2))*np.std(temp)

(0.5426163015911472, 4.570892158819163)